In [1]:
# Validate models

In [3]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
def create_target_columns_to_classification(df):
    target_cols = ['home_win_flag', 'draw_flag','away_win_flag']
    target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
    def get_y(row):
        for col in target_cols:
            if row[col] == 1:

                y = target_cols_dict[col]
        return y
    df['y'] = df[target_cols].apply(get_y,axis=1)
    df['y'] = df['y'].astype('category')
    return df

def get_pred_targets(x,colname):
    value = target_cols_dict[colname]
    a = 0
    if value == x:
        a = 1
    
    return a
target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
target_cols = ['home_win_flag','draw_flag','away_win_flag']

In [9]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]


df.head()
df = df.reset_index(drop=True)

target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

In [14]:
#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=34)
X_test

,home_team_match_nr,home_team_goal_roll1_sum,home_team_goal_roll2_sum,home_team_goal_roll3_sum,home_team_goal_roll4_sum,home_team_opponents_goal_roll1_sum,home_team_opponents_goal_roll2_sum,home_team_opponents_goal_roll3_sum,home_team_opponents_goal_roll4_sum,home_team_shot_roll1_sum,...,away_team_point_roll1_sum,away_team_point_roll2_sum,away_team_point_roll3_sum,away_team_point_roll4_sum,away_team_expected_point_roll1_sum,away_team_expected_point_roll2_sum,away_team_expected_point_roll3_sum,away_team_expected_point_roll4_sum,away_team_point_mean,away_team_expected_point_mean
62088,28.0,2.0,2.0,4.0,5.0,1.0,1.0,4.0,6.0,8.0,...,0.0,0.0,1.0,4.0,3.0,3.0,6.0,9.0,1.154,2.308
24707,14.0,1.0,2.0,4.0,6.0,1.0,3.0,4.0,5.0,10.0,...,3.0,6.0,7.0,10.0,3.0,6.0,9.0,12.0,2.333,0.500
46709,34.0,1.0,2.0,5.0,7.0,1.0,1.0,2.0,3.0,11.0,...,0.0,0.0,1.0,2.0,3.0,6.0,9.0,12.0,1.235,1.059
7094,13.0,1.0,4.0,7.0,9.0,2.0,5.0,5.0,6.0,7.0,...,1.0,1.0,1.0,2.0,0.0,0.0,3.0,6.0,1.200,0.000
47838,27.0,1.0,1.0,5.0,8.0,2.0,3.0,4.0,5.0,23.0,...,3.0,4.0,4.0,5.0,3.0,6.0,6.0,6.0,1.083,0.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41952,31.0,3.0,4.0,5.0,5.0,0.0,2.0,4.0,4.0,10.0,...,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,1.133,0.400
41130,22.0,0.0,2.0,2.0,5.0,2.0,3.0,5.0,5.0,5.0,...,1.0,1.0,1.0,2.0,3.0,6.0,6.0,9.0,1.455,2.727
2158,5.0,3.0,6.0,7.0,8.0,4.0,6.0,7.0,7.0,13.0,...,3.0,3.0,3.0,4.0,3.0,3.0,3.0,6.0,0.000,0.000
22138,10.0,0.0,2.0,3.0,3.0,2.0,3.0,4.0,6.0,6.0,...,3.0,3.0,6.0,7.0,0.0,0.0,3.0,3.0,0.500,0.000


In [12]:
# Load models

In [16]:
# load model
p_home_win = pickle.load(open('pipe_home_win_base_model.p', "rb"))
p_draw =     pickle.load(open('pipe_p_draw_base_model.p', "rb"))
p_away_win =  pickle.load(open('pipe_p_away_win_base_model.p', "rb"))
xgb_m =      pickle.load(open('xgb_m.p', "rb"))

In [21]:
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df = create_target_columns_to_classification(val_df)
retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(X_test.loc[retrain_index][features_cols])
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 0.8219446634437851
 draw_flag auc is: 0.6935300341192814
 away_win_flag auc is: 0.9580984641101086
